In [258]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from pandas import Series
warnings.filterwarnings('ignore')
%matplotlib inline

In [259]:
# Import the data
import json
dma = pd.read_csv("C:/AmyLe/Users/user/Desktop/Rutgers/DMA-zip.csv", encoding="iso-8859-1")
events = json.load(open('C:/AmyLe/Users/user/Desktop/Rutgers/events.json', 'r'))
#Convert list "event" to dataframe
events_df = pd.DataFrame(events)

In [260]:
# First look at the dma data
dma.info()
dma['ZIPCODE'] = dma['ZIPCODE'].apply(str)
dma['DMA CODE'] = dma['DMA CODE'].apply(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41535 entries, 0 to 41534
Data columns (total 6 columns):
FIPS        41535 non-null int64
COUNTY      41535 non-null object
ST          41535 non-null object
DMA CODE    41535 non-null int64
DMA NAME    41535 non-null object
ZIPCODE     41535 non-null int64
dtypes: int64(3), object(3)
memory usage: 1.9+ MB


In [261]:
dma.columns

Index(['FIPS', 'COUNTY', 'ST', 'DMA CODE', 'DMA NAME', 'ZIPCODE'], dtype='object')

In [262]:
dma.head(10)

,FIPS,COUNTY,ST,DMA CODE,DMA NAME,ZIPCODE
0,1001,Autauga,AL,698,MONTGOMERY,36003
1,1001,Autauga,AL,698,MONTGOMERY,36006
2,1001,Autauga,AL,698,MONTGOMERY,36008
3,1001,Autauga,AL,698,MONTGOMERY,36051
4,1001,Autauga,AL,698,MONTGOMERY,36066
5,1001,Autauga,AL,698,MONTGOMERY,36067
6,1001,Autauga,AL,698,MONTGOMERY,36068
7,1001,Autauga,AL,698,MONTGOMERY,36749
8,1003,Baldwin,AL,686,MOBILE-PENSACOLA,36507
9,1003,Baldwin,AL,686,MOBILE-PENSACOLA,36511


In [263]:
dma.rename(columns={"ZIPCODE": "venue_zip"}, inplace=True)

In [264]:
events_df.info()
events_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81474 entries, 0 to 81473
Data columns (total 16 columns):
event_description    81474 non-null object
event_end_utc        81474 non-null object
event_id             81474 non-null object
event_name           81474 non-null object
event_start_utc      81474 non-null object
facebook_event_id    53788 non-null object
hashtag              14287 non-null object
organization_id      81474 non-null object
organization_name    81474 non-null object
tags                 81474 non-null object
venue_city           81474 non-null object
venue_name           81474 non-null object
venue_state          81474 non-null object
venue_street         81474 non-null object
venue_timezone       81474 non-null object
venue_zip            81474 non-null object
dtypes: object(16)
memory usage: 9.9+ MB


Index(['event_description', 'event_end_utc', 'event_id', 'event_name',
       'event_start_utc', 'facebook_event_id', 'hashtag', 'organization_id',
       'organization_name', 'tags', 'venue_city', 'venue_name', 'venue_state',
       'venue_street', 'venue_timezone', 'venue_zip'],
      dtype='object')

In [265]:
# Filling missing values with "Unknown"
events_df=events_df.fillna(value="Unknown")

In [266]:
len(events_df_new['event_name'].unique())
### There must be some duplicates in the event name

72984

In [267]:
#I dropped all duplicate events across columns event name and state
events_df = events_df.drop_duplicates(subset=['event_name', 'venue_state'])

In [268]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73545 entries, 0 to 81473
Data columns (total 16 columns):
event_description    73545 non-null object
event_end_utc        73545 non-null object
event_id             73545 non-null object
event_name           73545 non-null object
event_start_utc      73545 non-null object
facebook_event_id    73545 non-null object
hashtag              73545 non-null object
organization_id      73545 non-null object
organization_name    73545 non-null object
tags                 73545 non-null object
venue_city           73545 non-null object
venue_name           73545 non-null object
venue_state          73545 non-null object
venue_street         73545 non-null object
venue_timezone       73545 non-null object
venue_zip            73545 non-null object
dtypes: object(16)
memory usage: 9.5+ MB


In [269]:
#Merge DMA name and code to events data
events_df = events_df.merge(dma[['DMA CODE','DMA NAME','venue_zip']],how='left',on='venue_zip')

In [270]:
events_df_new = events_df[['event_description', 'event_name','organization_name','venue_city', 'venue_state','DMA CODE','DMA NAME']]

In [279]:
#Create a new column called info which will contain event description; event name; state within each record:
events_df_new['info'] = events_df_new['event_description']+' '+events_df_new['event_name']+' '+events_df_new['organization_name']+' '+ events_df_new['DMA CODE']+' '+ events_df_new['venue_state'] 
events_df_new['info'].fillna('', inplace=True)
events_df_new.iloc[2]

event_description    Celebrate Halloween Regency style with this scary spin on a Jane Austen classic! Bring a picnic dinner and your best Halloween costume to take in this spooky movie on the lawn of Dumbarton House!\n\nWhat you need to know:\n\nMovie is rated PG-13 for zombie violence\n\nNo outside alcohol or glass\n\nPicnics welcome\n\nDress warm as event is outdoors\n\nWarm beverages and snacks available for purchase\n\nBlankets are encouraged. Visitors with chairs will be seated on the gravel path and at the back of the lawn.\n\nNo dogs and no smoking allowed\n\nTickets are non-refundable\n\nIn the event of rain, event will be canceled by 3pm via social media with a rain date of November 2nd\n\nGates open at 5:00pm for members and 5:30pm for the general public.\n\n\n                                                                            
event_name           Outdoor Movie Night: Pride and Prejudice and Zombies                                                                     

In [280]:
#TfidfVectorizer function from scikit-learn, which transforms text to feature vectors that can be used as input to estimator.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist


tfidf = TfidfVectorizer(input='content', encoding='iso-8859-1', 
                      decode_error='ignore', analyzer='word', max_df=0.8, # drop words that occur in more than 80% of events
                      min_df=5, # only use words that appear in at least 5 events 
                      lowercase=True, #convert everything to lower case
                      ngram_range=(1,2), stop_words='english')

In [281]:
tfidf_m = tfidf.fit_transform(events_df_new['info'])
print("Number of features: %d" % tfidf_m.get_shape()[1])
tfidf_m.shape

Number of features: 205434


(73545, 205434)

In [274]:
type(tfidf_m)

scipy.sparse.csr.csr_matrix

In [282]:
#Compute the SVD and convert data to latent space

import scipy
from scipy import linalg
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

U, sigma, Vt = svds(tfidf_m, k= 100)
U.shape

(73545, 100)

In [ ]:
"""
u, s, vt  = svds(tfidf_m)

energy_sq = scipy.linalg.norm(s)**2
approx_energy_sq = scipy.linalg.norm(s[:5])**2
percentage = approx_energy_sq*100/energy_sq
percentage

"""

In [283]:
U[0]

array([ 6.00581745e-03, -9.53591514e-04, -5.42287558e-03, -1.38631908e-03,
        5.56747245e-04, -8.03048402e-03, -6.88303019e-03,  1.05976269e-03,
       -1.07113579e-03, -5.46203629e-03,  4.29932310e-03, -3.13754451e-04,
        6.61954509e-04,  1.80356529e-03, -7.48142007e-04,  1.30517221e-03,
        2.21285443e-03,  1.65890476e-04, -2.75931846e-03, -2.86642884e-03,
        5.55294014e-03, -1.15914578e-03, -1.98711297e-03,  4.19428366e-03,
        9.19496259e-04,  4.90283276e-03,  2.15736545e-03, -7.49776700e-04,
       -1.63617353e-03, -4.93886365e-03, -2.07416954e-03,  2.53558518e-03,
       -4.19932100e-03,  4.48849431e-03,  3.19389358e-03,  7.69574048e-04,
       -1.95257751e-03, -1.09023767e-03,  1.44894133e-03,  2.15658085e-03,
       -1.64157561e-04,  2.77517436e-04, -7.95318984e-04,  2.62760994e-05,
       -4.09214190e-04, -4.58544319e-04, -1.70345745e-03,  3.27409213e-04,
        1.33071676e-03, -1.09836685e-03,  1.09995300e-03,  1.21205124e-03,
       -1.89879987e-03,  

I will be using the Cosine Similarity to calculate the similarity between two events. 
Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 

The next step is to write a function that returns the 5 most similar events based on the cosine similarity score.

In [284]:
event_name = events_df['event_name']
indices = pd.Series(events_df.index, index=events_df['event_name'])

from sklearn.metrics.pairwise import cosine_similarity
def events_recommendations(r):
    idx = indices[r]
    try:
        idx = idx.tolist()
        cos = cosine_similarity(U[idx[0]:int(idx[0])+1], U)
    except TypeError:
        cos = cosine_similarity(U[idx:int(idx)+1], U)
    sim_scores = list(enumerate(cos.ravel()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    event_indices = [i[0] for i in sim_scores]
    pd.set_option('display.max_colwidth', -1)
    return events_df[['event_name','venue_street','venue_city', 'venue_state','venue_zip','event_description']].iloc[event_indices]

In [228]:
"""
def events_recommendations(r):
    idx = indices[r]
    try:
        idx = idx.tolist()
        cos = cosine_similarity(U[idx[0]:int(idx[0])+1], U)
        sim_scores = list(enumerate(cos.ravel()))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        event_indices = [i[0] for i in sim_scores]
        pd.set_option('display.max_colwidth', -1)
        return events_df[['event_name','venue_street','venue_city', 'venue_state','venue_zip','event_description']].iloc[event_indices]
    except TypeError:
        cos = cosine_similarity(U[idx:int(idx)+1], U)
        sim_scores = list(enumerate(cos.ravel()))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        event_indices = [i[0] for i in sim_scores]
        pd.set_option('display.max_colwidth', -1)
        return events_df[['event_name','venue_street','venue_city', 'venue_state','venue_zip','event_description']].iloc[event_indices]
"""

In [285]:
#Testing
events_recommendations('Motobrewtour')

,event_name,venue_street,venue_city,venue_state,venue_zip,event_description
39636,Seveer Havoc,3100 Boston St,Baltimore,MD,21224,"Travis ""Seveer""Â Reeves defending his USBO title\n"
13367,Spring Bluegrass- Tom Beers Collective,12820 Long Green Pike,Hydes,MD,21082,Join this fabulous callobration of some of the finest Bluegrass musicians in MD.\n
57112,ANYONE CAN LOSE,6871 Dogwood Rd,Milford Mill,MD,21244,PROFESSIONAL BOXING\n
14341,Job Fair in Southern AACo,5757 Solomons Island Road,Lothian,MD,20711,Stop by and Visit Me at the Seacret Table!!!!\n\nÂ \n
35444,The Pickin' Coalition,12820 Long Green Pike,Hydes,MD,21082,The regions finest Bluegrass Bands\n


In [286]:
#Testing
events_recommendations('Evil Genius Yoga')

,event_name,venue_street,venue_city,venue_state,venue_zip,event_description
10206,EG Yoga,1727 N Front St,Philadelphia,PA,19122,"Swing by Sunday, June 10th forÂ yoga with Grace and Glory.\n\nAll levels are welcome - just bring a mat.Your class ticket includes your first 16oz beer after class.Â \n"
34265,Pints and Poses with Grace and Glory,1727 N Front St,Philadelphia,PA,19122,"Swing by Sunday, June 10th forÂ yoga with Grace and Glory.\n\nAll levels are welcome - just bring a mat.Your class ticket includes your first 16oz beer after class.Â \n"
1971,Yoga with Jake,1727 N Front St,Philadelphia,PA,19122,"Yoga with Jake!Enjoy a class, a glass & some snacks!$25 per person. All levels welcome - bring your own mat.Â \n"
50965,Zen Yoga with Devan-Yoga and a beer,324 W Main St,Lansdale,PA,19446,Join us for some Yoga in our Pub. Â $15 gets you one hour of Yoga plus a beer\n
46885,Beer & Yoga - Iron Hill Media,30 E State St,Media,PA,19063,"BEER & YOGA is back atÂ Iron Hill Brewery & RestaurantÂ in Media, PA!Join us on Saturday, June 16th for fun, all-levels yoga class led by Iron Hill's own Tori Tilley! After class, head on over to the bar for some healthy snacks and of course... BEER!Â Class will be from 9:30-10:30am, with food and drinks to follow!\n\n$25 to sign up - price includes the yoga class, a beer of your choice, and a spread of healthy snacks.Please bring your own yoga mat.No experience necessary - beginners welcome and encouraged!\n\nIf you have any questions, please callÂ (610) 627-9000 or email yogabytori@gmail.com!\n"


In [287]:
# In case the input is event_id.
event_id = events_df['event_id']
indices_id = pd.Series(events_df.index, index=events_df['event_id'])

from sklearn.metrics.pairwise import cosine_similarity

def event_recommendations(r):
    idx = indices_id[r]
    cos = cosine_similarity(U[idx:int(idx)+1], U)
    sim_scores = list(enumerate(cos.ravel()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    event_indices = [i[0] for i in sim_scores]
    pd.set_option('display.max_colwidth', -1)
    df = events_df[['event_id']].iloc[event_indices]
    df['query_id'] = r
    return df

In [288]:
event_recommendations('b8f8d1e2-f713-11e7-95bb-22000a66e11f')

,event_id,query_id
1909,9aee416a-f168-11e7-8513-22000aba37e5,b8f8d1e2-f713-11e7-95bb-22000a66e11f
5314,6f9349cc-29f2-11e7-9497-22000bb4eb03,b8f8d1e2-f713-11e7-95bb-22000a66e11f
38,bb13fcf8-2537-11e7-ba8a-22000bccc0a9,b8f8d1e2-f713-11e7-95bb-22000a66e11f
18844,ca41a59c-2625-11e7-ae7c-22000bb0d347,b8f8d1e2-f713-11e7-95bb-22000a66e11f
29121,5f90ea0a-c688-11e8-b57a-22000b0a8195,b8f8d1e2-f713-11e7-95bb-22000a66e11f


In [ ]:
"""
df = pd.DataFrame()
for r in event_id[0:1001]:
    df = df.append(event_recommendations(r))

"""

In [27]:
df.to_csv('phase4_first1000.csv', encoding='utf-8')

In [241]:
events_df.loc[events_df['event_name'] == 'Museum After Hours - August']

,event_description,event_end_utc,event_id,event_name,event_start_utc,facebook_event_id,hashtag,organization_id,organization_name,tags,venue_city,venue_name,venue_state,venue_street,venue_timezone,venue_zip,DMA CODE,DMA NAME
73543,"This is the secondÂ After Hours event at the museum. Adults over 21 only. Advance tickets only. Ticket priceÂ will include beer, wine and snacks.Â Get tours of the museum, hear stories about digging dinosaurs, try out interactive exhibits, maybe even play with dinosaur toys.\n",12 Aug 2018 02:00:00,86ebb60a-7a0e-11e8-a141-22000b2e0019,Museum After Hours - August,11 Aug 2018 23:00:00,Unknown,Unknown,earth-experience,Earth Experience,[],Murfreesboro,Earth Experience - Middle Tennessee Museum of Natural History,TN,816 Old Salem Rd,US/Central,37129,659,NASHVILLE
